In [1]:
from keras.losses import categorical_crossentropy, sparse_categorical_crossentropy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D, Flatten, Dropout
from keras.utils import to_categorical
import pandas as pd
import talib
from datetime import datetime as dt
import numpy as np
import Util

Using TensorFlow backend.


In [2]:
data_df, target_class = util.get_data(type = 'class')

type = classification


In [4]:
dataset = util.normalise_data(data_df)

In [5]:
TRAIN_SPLIT = round(0.8 * len(dataset))
past_history = 21
future_target = 1
STEP = 1
X_train, y_train =past_historyivariate_data(dataset, target_class, 0, TRAIN_SPLIT, past_history, future_target, STEP, single_step=True)
X_test, y_test = util.multivariate_data(dataset, target_class, TRAIN_SPLIT, None, past_history, future_target, STEP, single_step=True)
X_train = X_train.reshape(X_train.shape[0],21,21,1)
X_test = X_test.reshape(X_test.shape[0],21,21,1)
#X_train, y_train, X_test, y_test = util.train_test_data_for_cnn(dataset, target_class, 0, TRAIN_SPLIT, past_history, future_target, STEP, single_step=True)

In [7]:
model = Sequential()

In [8]:
model.add(Conv2D(32, (3, 3), padding="same", activation="relu", input_shape=(21, 21,1)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size =(2, 2)))
model.add(Flatten())
model.add(Dropout(0.25))
model.add(Dense(units = 128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(units = 64, activation = "relu"))
model.add(Dense(units=1, activation='sigmoid'))

In [9]:
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics = ["accuracy"])

In [10]:
BATCH_SIZE = 32
EPOCHS = 100

model.fit(X_train, y_train,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 1945 samples, validate on 469 samples
Epoch 1/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.7046 - accuracy: 0.5224 - val_loss: 0.6940 - val_accuracy: 0.4861
Epoch 2/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6914 - accuracy: 0.5090 - val_loss: 0.6968 - val_accuracy: 0.4968
Epoch 3/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6890 - accuracy: 0.5270 - val_loss: 0.6962 - val_accuracy: 0.5075
Epoch 4/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6873 - accuracy: 0.5332 - val_loss: 0.6945 - val_accuracy: 0.5096
Epoch 5/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6825 - accuracy: 0.5506 - val_loss: 0.7141 - val_accuracy: 0.5139
Epoch 6/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6765 - accuracy: 0.5830 - val_loss: 0.7609 - val_accuracy: 0.4819
Epoch 7/100
1945/1945 [==============================] - 4s 2ms/step - loss: 0.6714

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
plt.plot(model.predict(X_test), color = 'r')
plt.grid()
plt.plot(y_test)

In [ ]:
plt.plot(y_test)

In [ ]:
X_test = X_test.reshape(X_test.shape[0],21,21,1).shape

In [ ]:
data_df["temp"] = np.append([np.nan], data_df['close'][:-1],)
data_df = data_df.dropna()


